<a href="https://colab.research.google.com/github/deekshahs1817/AI-Projects/blob/main/dee_milestone_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip
!chmod +x ngrok
!mv ngrok /usr/local/bin/


--2025-12-02 16:27:00--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 35.71.179.82, 13.248.244.96, 75.2.60.68, ...
Connecting to bin.equinox.io (bin.equinox.io)|35.71.179.82|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13921656 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.28M  5.00MB/s    in 2.7s    

2025-12-02 16:27:04 (5.00 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13921656/13921656]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


In [ ]:
!ngrok version


ngrok version 2.3.41


In [ ]:
!ngrok authtoken 32YS1TCUPVxlJTjm0BU8jUUsNjj_4qTe4ynr13WXFkh43A3uT

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [ ]:
!pip install streamlit pytesseract easyocr opencv-python-headless fuzzywuzzy python-Levenshtein sentence-transformers scikit-learn matplotlib


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 30.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.3/153.3 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.7/180.7 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 978.2/978.2 kB 52.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.6/300.6 kB 20.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 62.8 MB/s eta 0:00:00


In [ ]:
!apt-get install -y tesseract-ocr


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 41 not upgraded.


In [ ]:
!apt-get install -y tesseract-ocr
!pip install streamlit opencv-python-headless numpy pillow matplotlib


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 41 not upgraded.


In [ ]:
%%writefile app.py
import streamlit as st
import sqlite3, hashlib, hmac, json, time, io, base64, math
from datetime import datetime, date
from PIL import Image
import numpy as np
import cv2
import pytesseract
from difflib import SequenceMatcher
import pandas as pd
import matplotlib.pyplot as plt
import re

# ==============================
# CONFIG
# ==============================
DB_PATH = "kyc_advanced.db"
SALT = b"KYC_SUPER_SALT_V1"

st.set_page_config(
    page_title="AI KYC Fraud System",
    page_icon="🛡️",
    layout="wide"
)

# ==============================
# DB INIT
# ==============================
def init_db():
    con = sqlite3.connect(DB_PATH)
    c = con.cursor()

    # Users
    c.execute("""
    CREATE TABLE IF NOT EXISTS users(
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        username TEXT UNIQUE,
        password_hash TEXT,
        role TEXT,
        created_at TEXT
    )
    """)

    # KYC records
    c.execute("""
    CREATE TABLE IF NOT EXISTS kycs(
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        username TEXT,
        claimed_name TEXT,
        claimed_phone TEXT,
        claimed_address TEXT,
        claimed_dob TEXT,
        aadhaar TEXT,
        pan TEXT,
        phone_extracted TEXT,
        dob_extracted TEXT,
        name_extracted TEXT,
        address_extracted TEXT,
        tamper_score REAL,
        blur_score REAL,
        name_score REAL,
        addr_score REAL,
        phone_match INTEGER,
        cnn_flag INTEGER,
        gnn_score REAL,
        rules_flag INTEGER,
        fraud_score REAL,
        risk_label TEXT,
        created_at TEXT
    )
    """)

    # Audits / alerts
    c.execute("""
    CREATE TABLE IF NOT EXISTS audits(
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        entry_id TEXT,
        payload TEXT,
        timestamp TEXT
    )
    """)

    # Fingerprints
    c.execute("""
    CREATE TABLE IF NOT EXISTS fingerprints(
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        account_id TEXT,
        descriptor BLOB,
        created_at TEXT
    )
    """)

    con.commit()
    con.close()

init_db()

# ==============================
# AUTH HELPERS
# ==============================
def hash_password(p: str) -> str:
    dk = hashlib.pbkdf2_hmac("sha256", p.encode(), SALT, 200000)
    return base64.b64encode(dk).decode()

def verify_password(p: str, stored: str) -> bool:
    dk = hashlib.pbkdf2_hmac("sha256", p.encode(), SALT, 200000)
    return hmac.compare_digest(base64.b64encode(dk).decode(), stored)

def create_user(username, password, role="user"):
    con = sqlite3.connect(DB_PATH)
    c = con.cursor()
    try:
        c.execute(
            "INSERT INTO users(username,password_hash,role,created_at) VALUES(?,?,?,?)",
            (username, hash_password(password), role,
             datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
        )
        con.commit()
        ok = True
    except Exception:
        ok = False
    con.close()
    return ok

def get_user(username):
    con = sqlite3.connect(DB_PATH)
    c = con.cursor()
    c.execute(
        "SELECT id,username,password_hash,role,created_at FROM users WHERE username=?",
        (username,)
    )
    row = c.fetchone()
    con.close()
    return row

def ensure_admin():
    con = sqlite3.connect(DB_PATH)
    c = con.cursor()
    c.execute("SELECT COUNT(*) FROM users WHERE role='admin'")
    cnt = c.fetchone()[0]
    con.close()
    if cnt == 0:
        create_user("admin", "admin123", "admin")

ensure_admin()

# ==============================
# AUDIT HELPERS
# ==============================
def save_audit(entry_id, payload_dict):
    con = sqlite3.connect(DB_PATH)
    c = con.cursor()
    c.execute(
        "INSERT INTO audits(entry_id,payload,timestamp) VALUES(?,?,?)",
        (entry_id, json.dumps(payload_dict),
         datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
    )
    con.commit()
    con.close()

def read_audits(n=50):
    con = sqlite3.connect(DB_PATH)
    c = con.cursor()
    c.execute("SELECT * FROM audits ORDER BY id DESC LIMIT ?", (n,))
    rows = c.fetchall()
    con.close()
    return rows

# ==============================
# OCR + EXTRACTION
# ==============================

AADHAAR_RE = re.compile(r"\d{4}\s?\d{4}\s?\d{4}")
PAN_RE = re.compile(r"[A-Z]{5}[0-9]{4}[A-Z]")
DOB_RE = re.compile(r"\d{2}[/-]\d{2}[/-]\d{4}")
PHONE_RE = re.compile(r"[6-9]\d{9}")
PIN_RE = re.compile(r"\b\d{6}\b")

def run_ocr(pil_img):
    try:
        text = pytesseract.image_to_string(pil_img)
    except Exception:
        text = ""
    return text

def extract_fields(text: str):
    lines = [l.strip() for l in text.splitlines() if l.strip()]

    # Name: first mostly alphabetic line
    name = "Not detected"
    for l in lines:
        stripped = l.replace(" ", "")
        if stripped.isalpha() and len(stripped) > 3:
            name = l.title()
            break

    aadhaar = "Not detected"
    m1 = AADHAAR_RE.search(text)
    if m1:
        aadhaar = m1.group()

    pan = "Not detected"
    m2 = PAN_RE.search(text)
    if m2:
        pan = m2.group()

    phone = "Not detected"
    m3 = PHONE_RE.search(text)
    if m3:
        phone = m3.group()

    dob = "Not detected"
    m4 = DOB_RE.search(text)
    if m4:
        dob = m4.group()

    address = "Not detected"
    pinm = PIN_RE.search(text)
    if pinm:
        p = pinm.group()
        for i, l in enumerate(lines):
            if p in l:
                address = " ".join(lines[max(0, i - 3): i + 1])
                break

    return {
        "name": name,
        "aadhaar": aadhaar,
        "pan": pan,
        "phone": phone,
        "dob": dob,
        "address": address
    }

# ==============================
# IMAGE FORENSICS (Tamper / Blur / Heatmap)
# ==============================
def tamper_blur_heat(pil_img):
    cv_img = cv2.cvtColor(np.array(pil_img), cv2.COLOR_RGB2BGR)
    gray = cv2.cvtColor(cv_img, cv2.COLOR_BGR2GRAY)

    edges = cv2.Canny(gray, 70, 150)
    tamper_score = float(np.mean(edges) / 255.0)  # 0–1

    blur = cv2.Laplacian(gray, cv2.CV_64F).var()

    heat = cv2.applyColorMap(edges, cv2.COLORMAP_JET)
    overlay = cv2.addWeighted(cv_img, 0.6, heat, 0.4, 0)
    _, buf = cv2.imencode(".png", overlay)
    heat_b64 = base64.b64encode(buf).decode()

    return tamper_score, blur, heat_b64

def cnn_tamper_flag(tamper_score):
    return 1 if tamper_score > 0.4 else 0

# ==============================
# EXIF GPS (Selfie)
# ==============================
def extract_exif_gps(pil_img):
    try:
        exif = pil_img._getexif()
        if not exif:
            return None
        gps = exif.get(34853)
        if not gps:
            return None

        def frac(v):
            return float(v[0]) / float(v[1])

        lat = frac(gps[2][0]) + frac(gps[2][1]) / 60 + frac(gps[2][2]) / 3600
        if gps[1] != "N":
            lat = -lat

        lon = frac(gps[4][0]) + frac(gps[4][1]) / 60 + frac(gps[4][2]) / 3600
        if gps[3] != "E":
            lon = -lon

        return {"lat": lat, "lon": lon}
    except Exception:
        return None

def age_from_dob(dob_str):
    m = re.search(r"(\d{2})[/-](\d{2})[/-](\d{4})", dob_str)
    if not m:
        return None
    d, mth, y = int(m.group(1)), int(m.group(2)), int(m.group(3))
    try:
        b = date(y, mth, d)
    except Exception:
        return None
    today = date.today()
    return today.year - b.year - ((today.month, today.day) < (b.month, b.day))

# ==============================
# SIMILARITY + GNN-LIKE NETWORK RISK
# ==============================
def sim_ratio(a, b):
    if not a or not b:
        return 0.0
    return SequenceMatcher(None, a.lower(), b.lower()).ratio()

def compute_gnn_score(aadhaar, phone):
    """Approximate GNN: check how many different users share same aadhaar/phone."""
    con = sqlite3.connect(DB_PATH)
    c = con.cursor()
    total_nodes = 0
    suspicious_degree = 0

    if aadhaar and aadhaar != "Not detected":
        c.execute("SELECT COUNT(DISTINCT username) FROM kycs WHERE aadhaar=?", (aadhaar,))
        cnt = c.fetchone()[0]
        total_nodes += cnt
        suspicious_degree += max(0, cnt - 1)

    if phone and phone != "Not detected":
        c.execute(
            "SELECT COUNT(DISTINCT username) FROM kycs WHERE phone_extracted=?",
            (phone,)
        )
        cnt2 = c.fetchone()[0]
        total_nodes += cnt2
        suspicious_degree += max(0, cnt2 - 1)

    con.close()
    if total_nodes == 0:
        return 0.0
    return min(1.0, suspicious_degree / max(1, total_nodes))

# ==============================
# AML RULE ENGINE
# ==============================
def aml_rules(username, aadhaar, phone, age):
    """Return (rules_flag, reasons list)."""
    reasons = []
    flag = 0
    con = sqlite3.connect(DB_PATH)
    c = con.cursor()

    # Duplicate Aadhaar
    if aadhaar and aadhaar != "Not detected":
        c.execute(
            "SELECT COUNT(DISTINCT username) FROM kycs WHERE aadhaar=?",
            (aadhaar,)
        )
        cnt = c.fetchone()[0]
        if cnt > 1:
            flag = 1
            reasons.append(f"Duplicate Aadhaar used by {cnt} users")

    # Duplicate phone
    if phone and phone != "Not detected":
        c.execute(
            "SELECT COUNT(DISTINCT username) FROM kycs WHERE phone_extracted=?",
            (phone,)
        )
        cnt = c.fetchone()[0]
        if cnt > 3:
            flag = 1
            reasons.append(f"Phone used in {cnt} user accounts")

    con.close()

    # Age rule
    if age is not None and age < 18:
        flag = 1
        reasons.append(f"Applicant age {age} (<18)")

    return flag, reasons

# ==============================
# FRAUD SCORE COMBINATION
# ==============================
def fraud_score(
    name_s, addr_s, phone_match,
    tamper, blur, cnn_flag,
    gnn_score, rules_flag
):
    score = 0
    score += name_s * 40.0
    score += addr_s * 20.0

    if phone_match:
        score += 10
    else:
        score -= 10

    if blur < 100:
        score -= 10

    score -= tamper * 20.0

    if cnn_flag:
        score -= 15

    score -= gnn_score * 20.0

    if rules_flag:
        score -= 20

    score = max(0, min(100, score))
    return score

def risk_label(score):
    if score >= 70:
        return "Low"
    if score >= 40:
        return "Medium"
    return "High"

# ==============================
# FINGERPRINT MODULE
# ==============================
def fp_des(cv_img):
    orb = cv2.ORB_create(500)
    gray = cv2.cvtColor(cv_img, cv2.COLOR_BGR2GRAY)
    _, des = orb.detectAndCompute(gray, None)
    return des

def store_fp(account_id, des):
    con = sqlite3.connect(DB_PATH)
    c = con.cursor()
    c.execute(
        "INSERT INTO fingerprints(account_id,descriptor,created_at) VALUES(?,?,?)",
        (account_id, des.tobytes(),
         datetime.now().strftime("%Y-%m-%d"))
    )
    con.commit()
    con.close()

def load_fp():
    con = sqlite3.connect(DB_PATH)
    c = con.cursor()
    c.execute("SELECT account_id,descriptor FROM fingerprints")
    rows = c.fetchall()
    con.close()
    out = []
    for acc, blob in rows:
        try:
            arr = np.frombuffer(blob, dtype=np.uint8).reshape((-1, 32))
            out.append((acc, arr))
        except Exception:
            continue
    return out

def fp_match(des1, des2):
    if des1 is None or des2 is None:
        return 0.0
    bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
    matches = bf.match(des1, des2)
    good = [m for m in matches if m.distance < 50]
    return len(good) / max(1, min(len(des1), len(des2)))

# ==============================
# SESSION INIT
# ==============================
if "logged_in" not in st.session_state:
    st.session_state["logged_in"] = False
if "username" not in st.session_state:
    st.session_state["username"] = None
if "role" not in st.session_state:
    st.session_state["role"] = None

# ==============================
# AUTH UI
# ==============================
def auth_ui():
    st.title("🔐 AI KYC Fraud Detection – Login / Register")
    tab1, tab2 = st.tabs(["Login", "Register"])

    with tab1:
        u = st.text_input("Username")
        p = st.text_input("Password", type="password")
        if st.button("Login"):
            row = get_user(u)
            if not row:
                st.error("Invalid username or password")
                return
            uid, uname, pwh, role, created = row
            if verify_password(p, pwh):
                st.session_state["logged_in"] = True
                st.session_state["username"] = uname
                st.session_state["role"] = role
                st.success(f"Welcome {uname} ({role})")
                st.rerun()
            else:
                st.error("Invalid username or password")

    with tab2:
        u2 = st.text_input("New username")
        p2 = st.text_input("New password", type="password")
        role = st.selectbox("Role", ["user", "admin"])
        if st.button("Create account"):
            if not u2 or not p2:
                st.error("Enter username & password")
            else:
                ok = create_user(u2, p2, role)
                if ok:
                    st.success("Account created, please login.")
                else:
                    st.error("Username already exists")

# ==============================
# KYC DASHBOARD
# ==============================
def kyc_dashboard():
    st.title("🛡️ KYC Verification Dashboard")

    st.write(f"Logged in as **{st.session_state['username']}** ({st.session_state['role']})")
    if st.button("Logout"):
        st.session_state["logged_in"] = False
        st.session_state["username"] = None
        st.session_state["role"] = None
        st.rerun()

    st.subheader("📄 Claimed Details")
    col1, col2 = st.columns(2)
    with col1:
        claimed_name = st.text_input("Full Name (Claimed)")
        claimed_phone = st.text_input("Phone (Claimed)")
        claimed_dob = st.text_input("DOB (dd/mm/yyyy, Claimed)")
    with col2:
        claimed_address = st.text_area("Address (Claimed)", height=90)

    st.subheader("📂 Upload Documents")
    docs = {}
    docs["Aadhaar"] = st.file_uploader(
        "Upload Aadhaar Image(s)",
        type=["jpg", "jpeg", "png"],
        accept_multiple_files=True
    )
    docs["PAN"] = st.file_uploader(
        "Upload PAN Image(s)",
        type=["jpg", "jpeg", "png"],
        accept_multiple_files=True
    )
    docs["Utility"] = st.file_uploader(
        "Upload Utility Bill Image(s)",
        type=["jpg", "jpeg", "png"],
        accept_multiple_files=True
    )
    docs["Selfie"] = st.file_uploader(
        "Upload Selfie Image(s)",
        type=["jpg", "jpeg", "png"],
        accept_multiple_files=True
    )

    if st.button("🔍 Run Full KYC Analysis"):
        if not claimed_name:
            st.error("Please enter claimed name.")
            return

        all_results = {}
        primary = None
        primary_type = None
        heatmaps = []

        # process non-selfie docs
        for dtype in ["Aadhaar", "PAN", "Utility"]:
            uploaded = docs[dtype]
            if not uploaded:
                continue
            entries = []
            for f in uploaded:
                pil = Image.open(f).convert("RGB")
                text = run_ocr(pil)
                fields = extract_fields(text)
                tamper, blur, heat = tamper_blur_heat(pil)
                entries.append({
                    "fields": fields,
                    "tamper": tamper,
                    "blur": blur,
                    "heat": heat
                })
                heatmaps.append((dtype, heat))
            all_results[dtype] = entries
            if primary is None and entries:
                primary = entries[0]["fields"]
                primary_type = dtype

        # selfie EXIF GPS (optional)
        selfie_gps = None
        if docs["Selfie"]:
            try:
                pil_self = Image.open(docs["Selfie"][0]).convert("RGB")
                selfie_gps = extract_exif_gps(pil_self)
            except Exception:
                selfie_gps = None

        if primary is None:
            st.error("No document uploaded to extract primary details.")
            return

        # similarity scores
        name_s = sim_ratio(claimed_name, primary["name"])
        addr_s = sim_ratio(claimed_address, primary.get("address", ""))
        phone_match = 1 if claimed_phone and claimed_phone == primary.get("phone", "") else 0

        # aggregate tamper/blur
        max_tamper = 0.0
        min_blur = None
        for dtype, entries in all_results.items():
            for e in entries:
                max_tamper = max(max_tamper, e["tamper"])
                if min_blur is None:
                    min_blur = e["blur"]
                else:
                    min_blur = min(min_blur, e["blur"])
        if min_blur is None:
            min_blur = 0.0

        cnn_flag = cnn_tamper_flag(max_tamper)
        gnn_s = compute_gnn_score(
            primary.get("aadhaar", ""),
            primary.get("phone", "")
        )
        age = age_from_dob(claimed_dob)
        rules_flag, reasons = aml_rules(
            st.session_state["username"],
            primary.get("aadhaar", ""),
            primary.get("phone", ""),
            age
        )

        fscore = fraud_score(
            name_s, addr_s, phone_match,
            max_tamper, min_blur,
            cnn_flag, gnn_s, rules_flag
        )
        rlab = risk_label(fscore)

        # Save to DB
        con = sqlite3.connect(DB_PATH)
        c = con.cursor()
        c.execute("""
        INSERT INTO kycs(
            username, claimed_name, claimed_phone, claimed_address, claimed_dob,
            aadhaar, pan, phone_extracted, dob_extracted, name_extracted, address_extracted,
            tamper_score, blur_score, name_score, addr_score, phone_match,
            cnn_flag, gnn_score, rules_flag, fraud_score, risk_label, created_at
        ) VALUES(?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)
        """, (
            st.session_state["username"], claimed_name, claimed_phone,
            claimed_address, claimed_dob,
            primary.get("aadhaar", ""), primary.get("pan", ""),
            primary.get("phone", ""), primary.get("dob", ""),
            primary.get("name", ""), primary.get("address", ""),
            max_tamper, min_blur, name_s, addr_s, phone_match,
            cnn_flag, gnn_s, rules_flag, fscore, rlab,
            datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        ))
        con.commit()
        con.close()

        payload = {
            "user": st.session_state["username"],
            "primary_doc": primary_type,
            "primary_fields": primary,
            "name_similarity": name_s,
            "addr_similarity": addr_s,
            "phone_match": phone_match,
            "tamper": max_tamper,
            "blur": min_blur,
            "cnn_flag": cnn_flag,
            "gnn_score": gnn_s,
            "rules_flag": rules_flag,
            "aml_reasons": reasons,
            "fraud_score": fscore,
            "risk_label": rlab,
            "selfie_gps": selfie_gps
        }
        save_audit(f"kyc_{time.time()}", payload)

        # Display result
        st.subheader("✅ KYC Result")
        risk_emoji = "🟩" if rlab == "Low" else "🟧" if rlab == "Medium" else "🟥"
        st.metric("Fraud Risk", f"{risk_emoji} {rlab}", f"{fscore:.2f}")
        st.write("**Name Match:**", f"{name_s * 100:.1f}%")
        st.write("**Address Match:**", f"{addr_s * 100:.1f}%")
        st.write("**Phone Match:**", "Yes" if phone_match else "No")
        st.write("**Tamper Score (0–1):**", f"{max_tamper:.3f}")
        st.write("**Blur Score:**", f"{min_blur:.1f}")
        st.write("**CNN Tamper Flag:**", "Suspicious" if cnn_flag else "Clean")
        st.write("**Network (GNN-like) Risk Score:**", f"{gnn_s:.2f}")
        st.write("**AML Rules Triggered:**", "Yes" if rules_flag else "No")

        if reasons:
            st.subheader("⚠️ AML / Rule-based Reasons")
            for r in reasons:
                st.write("•", r)

        if selfie_gps:
            st.subheader("📍 Selfie EXIF GPS")
            st.json(selfie_gps)

        st.subheader("📸 Heatmap Evidence (Tamper Regions)")
        if heatmaps:
            for doc_name, hb64 in heatmaps:
                st.write(f"**{doc_name}**")
                try:
                    st.image(base64.b64decode(hb64), use_container_width=True)
                except Exception:
                    st.write("Unable to render heatmap.")
        else:
            st.info("No heatmaps available.")

        st.success("KYC Verification recorded in system.")

    st.markdown("---")
    st.subheader("📊 Your Recent KYC Records")
    con = sqlite3.connect(DB_PATH)
    df = pd.read_sql_query(
        "SELECT * FROM kycs WHERE username=? ORDER BY id DESC LIMIT 20",
        con, params=(st.session_state["username"],)
    )
    con.close()
    if df.empty:
        st.info("No KYC records yet.")
    else:
        st.dataframe(df)
        fig, ax = plt.subplots()
        df["risk_label"].value_counts().plot(kind="pie", autopct="%1.1f%%", ax=ax)
        ax.set_ylabel("")
        ax.set_title("Risk Level Distribution")
        st.pyplot(fig)

# ==============================
# FRAUD ANALYTICS DASHBOARD
# ==============================
def analytics_dashboard():
    st.title("📊 Global Fraud Analytics")

    con = sqlite3.connect(DB_PATH)
    df = pd.read_sql_query("SELECT * FROM kycs ORDER BY id DESC", con)
    con.close()

    if df.empty:
        st.info("No data yet.")
        return

    col1, col2, col3, col4 = st.columns(4)
    col1.metric("Total KYCs", len(df))
    col2.metric("High Risk", int((df["risk_label"] == "High").sum()))
    col3.metric("Medium Risk", int((df["risk_label"] == "Medium").sum()))
    col4.metric("Low Risk", int((df["risk_label"] == "Low").sum()))

    st.subheader("Fraud Score Distribution")
    fig1, ax1 = plt.subplots()
    df["fraud_score"].plot(kind="hist", bins=10, ax=ax1)
    ax1.set_xlabel("Fraud Score")
    ax1.set_title("Fraud Score Histogram")
    st.pyplot(fig1)

    df["created_at_dt"] = pd.to_datetime(df["created_at"])
    ts = df.groupby(df["created_at_dt"].dt.date)["fraud_score"].mean()
    st.subheader("Average Fraud Score Over Time")
    fig2, ax2 = plt.subplots()
    ts.plot(kind="line", marker="o", ax=ax2)
    ax2.set_xlabel("Date")
    ax2.set_ylabel("Average Fraud Score")
    st.pyplot(fig2)

    st.subheader("Common Fraud Indicators")
    indicators = {
        "CNN Tamper Flag": int(df["cnn_flag"].sum()),
        "AML Rules Triggered": int(df["rules_flag"].sum()),
        "High Network Risk (gnn_score>0.5)": int((df["gnn_score"] > 0.5).sum())
    }
    fig3, ax3 = plt.subplots()
    pd.Series(indicators).plot(kind="bar", ax=ax3)
    ax3.set_ylabel("Count")
    st.pyplot(fig3)

    st.subheader("Suspicious Identity Clusters (GNN-like)")
    dup_aadhaar = df[df["aadhaar"].notna() & (df["aadhaar"] != "Not detected")]
    group = dup_aadhaar.groupby("aadhaar")["username"].nunique()
    suspicious = group[group > 1]
    if suspicious.empty:
        st.info("No suspicious Aadhaar clusters.")
    else:
        for aadhaar, count in suspicious.items():
            st.write(f"• Aadhaar `{aadhaar}` used by {count} users")

# ==============================
# ALERTS
# ==============================
def alerts_view():
    st.title("🔔 Live Fraud Alerts")

    if st.button("Refresh Alerts"):
        st.rerun()

    rows = read_audits(30)
    if not rows:
        st.info("No alerts yet.")
        return

    for r in rows:
        _id, entry_id, payload, ts = r
        try:
            data = json.loads(payload)
        except Exception:
            data = {"raw": payload}
        risk = data.get("risk_label") or "Medium"
        emoji = "🟩"
        if risk == "High":
            emoji = "🟥"
        elif risk == "Medium":
            emoji = "🟧"
        st.markdown(f"**{emoji} {risk}** — {ts}")
        with st.expander("Details"):
            st.json(data)

# ==============================
# ADMIN PANEL
# ==============================
def admin_panel():
    st.title("🧑‍💼 Admin Panel")

    con = sqlite3.connect(DB_PATH)
    users_df = pd.read_sql_query(
        "SELECT id,username,role,created_at FROM users ORDER BY id",
        con
    )
    kycs_df = pd.read_sql_query("SELECT * FROM kycs ORDER BY id DESC", con)
    con.close()

    st.subheader("Users")
    st.dataframe(users_df)
    col1, col2, col3 = st.columns(3)
    with col1:
        u = st.text_input("Username to make admin")
        if st.button("Make Admin"):
            if u:
                con = sqlite3.connect(DB_PATH)
                c = con.cursor()
                c.execute("UPDATE users SET role='admin' WHERE username=?", (u,))
                con.commit()
                con.close()
                st.success(f"{u} is now admin")
                st.rerun()
    with col2:
        u2 = st.text_input("Username to make user")
        if st.button("Make User"):
            if u2:
                con = sqlite3.connect(DB_PATH)
                c = con.cursor()
                c.execute("UPDATE users SET role='user' WHERE username=?", (u2,))
                con.commit()
                con.close()
                st.success(f"{u2} is now user")
                st.rerun()
    with col3:
        u3 = st.text_input("Username to delete")
        if st.button("Delete User"):
            if u3:
                con = sqlite3.connect(DB_PATH)
                c = con.cursor()
                c.execute("DELETE FROM users WHERE username=?", (u3,))
                con.commit()
                con.close()
                st.warning(f"Deleted {u3}")
                st.rerun()

    st.markdown("---")
    st.subheader("All KYC Records")
    st.dataframe(kycs_df)
    csv = kycs_df.to_csv(index=False).encode("utf-8")
    st.download_button("Download KYC CSV", csv, "kyc_records.csv")

# ==============================
# FINGERPRINT PAGE
# ==============================
def fingerprint_page():
    st.title("🖐️ Fingerprint Enrollment & Matching")

    acc = st.text_input("Account ID")
    enroll = st.checkbox("Enroll uploaded fingerprint")
    fp_file = st.file_uploader(
        "Upload fingerprint image",
        type=["jpg", "jpeg", "png"]
    )

    if fp_file and st.button("Process Fingerprint"):
        pil = Image.open(fp_file).convert("RGB")
        cv_img = cv2.cvtColor(np.array(pil), cv2.COLOR_RGB2BGR)
        des = fp_des(cv_img)
        if enroll:
            if not acc:
                st.error("Enter account id to enroll.")
                return
            store_fp(acc, des)
            st.success("Fingerprint enrolled.")

        stored = load_fp()
        best_score = 0.0
        best_acc = None
        for sacc, sdes in stored:
            s = fp_match(des, sdes)
            if s > best_score:
                best_score = s
                best_acc = sacc

        st.metric("Best match score", f"{best_score:.2f}")
        if best_score > 0.35:
            st.success(f"Matched with account {best_acc}")
        else:
            st.info("No strong match found.")

# ==============================
# MAIN ROUTING
# ==============================
def main():
    if not st.session_state["logged_in"]:
        auth_ui()
        return

    st.sidebar.title("Navigation")
    options = ["KYC Dashboard", "Fraud Analytics", "Alerts", "Fingerprint", "Admin", "Logout"]
    if st.session_state["role"] != "admin":
        if "Admin" in options:
            options.remove("Admin")
    choice = st.sidebar.radio("Go to", options)

    if choice == "KYC Dashboard":
        kyc_dashboard()
    elif choice == "Fraud Analytics":
        analytics_dashboard()
    elif choice == "Alerts":
        alerts_view()
    elif choice == "Fingerprint":
        fingerprint_page()
    elif choice == "Admin":
        admin_panel()
    elif choice == "Logout":
        st.session_state["logged_in"] = False
        st.session_state["username"] = None
        st.session_state["role"] = None
        st.rerun()

if __name__ == "__main__":
    main()


Overwriting app.py


In [ ]:
!apt-get install -y tesseract-ocr
!pip install streamlit opencv-python-headless numpy pillow matplotlib


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 41 not upgraded.


In [ ]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip
!mv ngrok /usr/local/bin/
!ngrok authtoken 32YS1TCUPVxlJTjm0BU8jUUsNjj_4qTe4ynr13WXFkh43A3uT


--2025-12-02 16:38:20--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 35.71.179.82, 13.248.244.96, 75.2.60.68, ...
Connecting to bin.equinox.io (bin.equinox.io)|35.71.179.82|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13921656 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip.3’

ngrok-stable-linux- 100%[===================>]  13.28M  5.37MB/s    in 2.5s    

2025-12-02 16:38:24 (5.37 MB/s) - ‘ngrok-stable-linux-amd64.zip.3’ saved [13921656/13921656]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   
Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [ ]:
!streamlit run app.py --server.port 8501 --server.enableCORS false --server.enableXsrfProtection false &




2025-12-02 17:35:56.328 Port 8501 is already in use


In [ ]:
from pyngrok import ngrok

# Replace with your own authtoken
ngrok.set_auth_token("32YS1TCUPVxlJTjm0BU8jUUsNjj_4qTe4ynr13WXFkh43A3uT")

In [ ]:
!curl ipv4.icanhazip.com

34.81.114.0


In [ ]:
!streamlit run app.py --server.port 8501 --server.address 0.0.0.0 &>/dev/null &

In [ ]:
from pyngrok import ngrok

ngrok.set_auth_token("32YS1TCUPVxlJTjm0BU8jUUsNjj_4qTe4ynr13WXFkh43A3uT")  # put your token here
public_url = ngrok.connect(8501)
print("Public URL:", public_url)

Public URL: NgrokTunnel: "https://b9b2d0bbf92e.ngrok-free.app" -> "http://localhost:8501"
